In [672]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [673]:
file_version = 122
CFG = {
    'EPOCHS': 30,
    'BATCH_SIZE':256,  #256  128  512
    'SEED':1,     #43
    'log_transform_turn': 1,  # 로그 트랜스폼을 몇번 돌릴 것인가?
    'skew_cut':0,           # 왜도값 얼마 초과시 로그 트랜스폼 할 것인가?
    'vif_cut': 10,           # 다중공선성 vif 얼마 이상을 제거할 것인가?
    'corr_cut': [0.045, 0.045, 0.045, 0.05],     # 일정 상관관계 이하 칼럼은 드랍
    
    'drop_rate': 0.3,     # overfit을 해소하기 위한 nn model dropout 비율 (0.1 ~ 0.5)  
    'reduct' : 'mean',      # BCELoss function reduction option : sum, mean
    'T_LEARNING_RATE': [3e-2, 1e-3, 1e-2, 3e-2],    # 0.01
    'T_Thresh': [0.3, 0.3, 0.3, 0.3],   # Teacher model train Threshhold 최초 0.35
    'S_LEARNING_RATE': [3e-2, 1e-3, 1e-2, 3e-2],    # 0.01
    'S_Thresh': [0.3, 0.3, 0.3, 0.3],     # Student model train Threshold 최초 0.35
    'alpha': 0.1   # distillation loss function 반영 비율
}
scaler = StandardScaler()  # StandardScaler, MinMaxScaler, RobustScaler

In [674]:
import pickle
#불러오기

with open(f"./data/{file_version}_1_train_X", 'rb') as pickle_filename:
    train_X_1 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_1_train_y", 'rb') as pickle_filename:
    train_y_1 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_1_val_X", 'rb') as pickle_filename:
    val_X_1 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_1_val_y", 'rb') as pickle_filename:
    val_y_1 = pickle.load(pickle_filename)     
with open(f"./data/{file_version}_1_test", 'rb') as pickle_filename:
    test_1 = pickle.load(pickle_filename)

with open(f"./data/{file_version}_2_train_X", 'rb') as pickle_filename:
    train_X_2 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_2_train_y", 'rb') as pickle_filename:
    train_y_2 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_2_val_X", 'rb') as pickle_filename:
    val_X_2 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_2_val_y", 'rb') as pickle_filename:
    val_y_2 = pickle.load(pickle_filename)     
with open(f"./data/{file_version}_2_test", 'rb') as pickle_filename:
    test_2 = pickle.load(pickle_filename)

with open(f"./data/{file_version}_3_train_X", 'rb') as pickle_filename:
    train_X_3 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_3_train_y", 'rb') as pickle_filename:
    train_y_3 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_3_val_X", 'rb') as pickle_filename:
    val_X_3 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_3_val_y", 'rb') as pickle_filename:
    val_y_3 = pickle.load(pickle_filename)     
with open(f"./data/{file_version}_3_test", 'rb') as pickle_filename:
    test_3 = pickle.load(pickle_filename)
    
with open(f"./data/{file_version}_4_train_X", 'rb') as pickle_filename:
    train_X_4 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_4_train_y", 'rb') as pickle_filename:
    train_y_4 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_4_val_X", 'rb') as pickle_filename:
    val_X_4 = pickle.load(pickle_filename)
with open(f"./data/{file_version}_4_val_y", 'rb') as pickle_filename:
    val_y_4 = pickle.load(pickle_filename)     
with open(f"./data/{file_version}_4_test", 'rb') as pickle_filename:
    test_4 = pickle.load(pickle_filename)

In [675]:
train_X_list = [train_X_1, train_X_2, train_X_3, train_X_4]
train_y_list = [train_y_1, train_y_2, train_y_3, train_y_4]
val_X_list = [val_X_1, val_X_2, val_X_3, val_X_4]
val_y_list = [val_y_1, val_y_2, val_y_3, val_y_4]
test_list = [test_1, test_2, test_3, test_4]

In [676]:
train_X_1.shape, train_y_1.shape, val_X_1.shape, val_y_1.shape

((3112, 27), (3112,), (778, 27), (778,))

In [677]:
train_X_2.shape, train_y_2.shape, val_X_2.shape, val_y_2.shape

((1852, 22), (1852,), (464, 22), (464,))

In [678]:
train_X_3.shape, train_y_3.shape, val_X_3.shape, val_y_3.shape

((5640, 28), (5640,), (1410, 28), (1410,))

In [679]:
train_X_4.shape, train_y_4.shape, val_X_4.shape, val_y_4.shape

((671, 24), (671,), (168, 24), (168,))

In [680]:
# train_X_1.columns.tolist()

In [681]:
class CustomDataset(Dataset):
    def __init__(self, data_X, data_y, distillation=False):
        super(CustomDataset, self).__init__()
        self.data_X = data_X
        self.data_y = data_y
        self.distillation = distillation
        
    def __len__(self):
        return len(self.data_X)
    
    def __getitem__(self, index):
        if self.distillation:
            # 지식 증류 학습 시
            teacher_X = torch.Tensor(self.data_X.iloc[index])
            student_X = torch.Tensor(self.data_X[test_stage_features].iloc[index])
            y = self.data_y.values[index]
            return teacher_X, student_X, y
        else:
            if self.data_y is None:
                test_X = torch.Tensor(self.data_X.iloc[index])
                return test_X
            else:
                teacher_X = torch.Tensor(self.data_X.iloc[index])

                y = self.data_y.values[index]
                return teacher_X, y

In [682]:
train_loaders = []
val_loaders = []

for a, b, c, d, e in zip(train_X_list, train_y_list, val_X_list, val_y_list, test_list):
    
    test_stage_features = e.iloc[:,:-1].columns
    
    train_dataset = CustomDataset(a, b, False)
    val_dataset = CustomDataset(c, d, False)
    
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)
    
#     print(train_loader)
#     print(list(train_loader))
    
    train_loaders.append(train_loader)
    val_loaders.append(val_loader)


In [683]:
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=t_start_col_num, out_features=256),
            nn.BatchNorm1d(256),
            nn.Dropout(p=CFG['drop_rate']),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(p=CFG['drop_rate']),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=256),
            nn.BatchNorm1d(256),
            nn.Dropout(p=CFG['drop_rate']),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=1),
            nn.Dropout(p=CFG['drop_rate']),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

In [684]:
def train(model, optimizer, train_loader, val_loader, scheduler, device, t_thresh):
    model.to(device)

    best_score = 0
    best_model = None
    
    # Loss Function 정의
    criterion = nn.BCELoss(reduction = CFG['reduct']).to(device)  #BCEWithLogitsLoss
#     criterion = nn.BCEWithLogitsLoss(reduction = CFG['reduct']).to(device)  #BCEWithLogitsLoss

    for epoch in range(CFG["EPOCHS"]):
        train_loss = []
  
        model.train()
        for X, y in tqdm(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            
            optimizer.zero_grad()
            
            y_pred = model(X)
            
            loss = criterion(y_pred, y.reshape(-1, 1))  # loss_fn(input, target)
            loss.backward()
            
            optimizer.step()

            train_loss.append(loss.item())

        val_loss, val_score = validation_teacher(model, val_loader, criterion, device, t_thresh)
        
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')

        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = model
            best_score = val_score
    print(f"Teacher Train Best Score는 {best_score} 입니다.")
    return best_model, best_score

In [685]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")


def validation_teacher(model, val_loader, criterion, device, t_thresh):
    model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = t_thresh     
    
    with torch.no_grad():
        for X, y in tqdm(val_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            
            model_pred = model(X.to(device))
            
            loss = criterion(model_pred, y.reshape(-1, 1))
            val_loss.append(loss.item())      
            
            model_pred = model_pred.squeeze(1).to('cpu')  # squeeze함수는 차원이 1인 차원을 제거해준다.
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        #print(pred_labels[0:10])
        # 오일상태는 0: 정상, 1: 이상
        # 어느 임계점 thrshold보다 큰 예측 라벨을 1로 하고, 나머지를 0으로 할 것인가?
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        #print(pred_labels[0:10])
        cf = confusion_matrix(true_labels, pred_labels)
        val_f1 = competition_metric(true_labels, pred_labels)
        
        print(cf)
        print(classification_report(true_labels, pred_labels, target_names=["0: 정상", "1: 이상"]))
    return val_loss, val_f1

In [686]:
teachers = []
results = []
num = 0

for a, b, c, d in zip(train_loaders, val_loaders, train_X_list, test_list):
    t_start_col_num = len(c.columns)
    test_stage_features = d.iloc[:,:-1].columns
#     s_start_col_num = len(d.columns)
    
    t_thresh = CFG["T_Thresh"][num]
    
    model = Teacher()
    model.eval()
    optimizer = torch.optim.Adam(model.parameters(), lr=CFG['T_LEARNING_RATE'][num])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)
    pred_labels = []
    true_labels = []

    teacher_model = train(model, optimizer, a, b, scheduler, device, t_thresh)
    m, s = teacher_model
    teachers.append(m)
    results.append(s)
    
    print("부분학습 종료")
    
    num += 1
       
    print("***"*50)

    
print("전체 학습 종료")
print(f"Best_Scores : {results}")

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

[[540 176]
 [ 11  51]]
              precision    recall  f1-score   support

       0: 정상       0.98      0.75      0.85       716
       1: 이상       0.22      0.82      0.35        62

    accuracy                           0.76       778
   macro avg       0.60      0.79      0.60       778
weighted avg       0.92      0.76      0.81       778

Epoch [0], Train Loss : [0.53563] Val Loss : [0.36056] Val F1 Score : [0.60267]


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

[[148  10]
 [  8   2]]
              precision    recall  f1-score   support

       0: 정상       0.95      0.94      0.94       158
       1: 이상       0.17      0.20      0.18        10

    accuracy                           0.89       168
   macro avg       0.56      0.57      0.56       168
weighted avg       0.90      0.89      0.90       168

Epoch [28], Train Loss : [0.29254] Val Loss : [0.17044] Val F1 Score : [0.56225]
Epoch 00029: reducing learning rate of group 0 to 3.6621e-06.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[[148  10]
 [  8   2]]
              precision    recall  f1-score   support

       0: 정상       0.95      0.94      0.94       158
       1: 이상       0.17      0.20      0.18        10

    accuracy                           0.89       168
   macro avg       0.56      0.57      0.56       168
weighted avg       0.90      0.89      0.90       168

Epoch [29], Train Loss : [0.27269] Val Loss : [0.17036] Val F1 Score : [0.56225]
Teacher Train Best Score는 0.6807680768076807 입니다.
부분학습 종료
******************************************************************************************************************************************************
전체 학습 종료
Best_Scores : [0.8356817427511087, 0.8494288681204569, 0.8193002691272587, 0.6807680768076807]


-Best_Scores : [0.8358817325031472, 0.7667412068576975, 0.8331679389312977, 0.6248138957816377]


In [687]:
for i, teacher in enumerate(teachers):
    PATH1 = f"./models/{file_version}_{i}_teacher_net.pth"
    torch.save(teacher.state_dict(), PATH1)

In [688]:
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=s_start_col_num, out_features=128),
            nn.BatchNorm1d(128),
            nn.Dropout(p=CFG['drop_rate']),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=512),
            nn.BatchNorm1d(512),
            nn.Dropout(p=CFG['drop_rate']),
            nn.LeakyReLU(),
            nn.Linear(in_features=512, out_features=128),
            nn.BatchNorm1d(128),
            nn.Dropout(p=CFG['drop_rate']),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=1),
            nn.Dropout(p=CFG['drop_rate']),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

In [689]:
# distillation Function 정의

# def distillation(student_logits, labels, teacher_logits, alpha, T):
def distillation(student_logits, labels, teacher_logits, alpha, T):

    student_loss = nn.BCELoss(reduction = CFG['reduct'])(student_logits, labels.reshape(-1, 1)) #BCEWithLogitsLoss
    
#     distillation_loss = nn.BCELoss()(student_logits, teacher_logits)
    distillation_loss = nn.KLDivLoss(reduction='batchmean')(F.log_softmax(student_logits/T, dim=1), F.softmax(teacher_logits/T, dim=1)) #* (T * T)
    
    return alpha * student_loss + (1-alpha) * distillation_loss

def distill_loss(output, target, teacher_output, loss_fn=distillation, opt=optimizer):
    loss_b = loss_fn(output, target, teacher_output, alpha=0.1, T=10)
#     loss_b = loss_fn(output, target, teacher_output, alpha=CFG['alpha'])

    if opt is not None:
        opt.zero_grad()
        loss_b.backward()
        opt.step()

    return loss_b.item()

In [690]:
def student_train(s_model, t_model, optimizer, train_loader, val_loader, scheduler, device, s_thresh):
    s_model.to(device)
    t_model.to(device)
    
    best_score = 0
    best_model = None

    for epoch in range(CFG["EPOCHS"]):
        train_loss = []
        s_model.train()
        t_model.eval()    # teather는 이번에는 학습하지 않으므로 .eval()로 불러온다
        
        for X_t, X_s, y in tqdm(train_loader):
            X_t = X_t.float().to(device)
            X_s = X_s.float().to(device)
            y = y.float().to(device)
            
            optimizer.zero_grad()

            output = s_model(X_s)
            with torch.no_grad():
                teacher_output = t_model(X_t)
                
            loss_b = distill_loss(output, y, teacher_output, loss_fn=distillation, opt=optimizer)

            train_loss.append(loss_b)

        val_loss, val_score = validation_student(s_model, t_model, val_loader, distill_loss, device, s_thresh)
        
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = s_model
            best_score = val_score
    
    print(f"Student Train Best Score는 {best_score} 입니다.")
        
    return best_model, best_score

In [691]:
def validation_student(s_model, t_model, val_loader, criterion, device, s_thresh):
    s_model.eval()
    t_model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = s_thresh
    
    with torch.no_grad():
        for X_t, X_s, y in tqdm(val_loader):
            X_t = X_t.float().to(device)
            X_s = X_s.float().to(device)
            y = y.float().to(device)
            
            model_pred = s_model(X_s)
            teacher_output = t_model(X_t)
            
            loss_b = distill_loss(model_pred, y, teacher_output, loss_fn=distillation, opt=None)
            val_loss.append(loss_b)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        val_f1 = competition_metric(true_labels, pred_labels)
        cf = confusion_matrix(true_labels, pred_labels)        
        print(cf)
        print(classification_report(true_labels, pred_labels, target_names=["0: 정상", "1: 이상"]))
        
    return val_loss, val_f1    

In [692]:
s_train_loaders = []
s_val_loaders = []

for a, b, c, d, e in zip(train_X_list, train_y_list, val_X_list, val_y_list, test_list):
    
    test_stage_features = e.iloc[:,:-1].columns
    
    train_dataset = CustomDataset(a, b, True)
    val_dataset = CustomDataset(c, d, True)
    
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)
    
    
    s_train_loaders.append(train_loader)
    s_val_loaders.append(val_loader)


In [693]:
students = []
results = []
num = 0

for a, b, c, d, e in zip(s_train_loaders, s_val_loaders, train_X_list, test_list, teachers):
    
    t_start_col_num = len(c.columns)
    test_stage_features = d.iloc[:,:-1].columns
    s_start_col_num = len(test_stage_features)
    s_thresh = CFG["S_Thresh"][num]

    student_model = Student()
    student_model.eval()
    optimizer = torch.optim.Adam(student_model.parameters(), lr=CFG['S_LEARNING_RATE'][num])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)

    best_student_model = student_train(student_model, e, optimizer, a, b, scheduler, device, s_thresh)
    
    m, s = best_student_model

    students.append(m)
    results.append(s)
    
    num += 1
    
    print("부분학습 종료")

    print("-"*70)
    
print("전체 학습 종료")
print(f"best scores : {results}")
# print(students)

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

[[650  66]
 [ 52  10]]
              precision    recall  f1-score   support

       0: 정상       0.93      0.91      0.92       716
       1: 이상       0.13      0.16      0.14        62

    accuracy                           0.85       778
   macro avg       0.53      0.53      0.53       778
weighted avg       0.86      0.85      0.86       778

Epoch [0], Train Loss : [0.05373] Val Loss : [0.03125] Val F1 Score : [0.53086]


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

[[152   6]
 [ 10   0]]
              precision    recall  f1-score   support

       0: 정상       0.94      0.96      0.95       158
       1: 이상       0.00      0.00      0.00        10

    accuracy                           0.90       168
   macro avg       0.47      0.48      0.47       168
weighted avg       0.88      0.90      0.89       168

Epoch [28], Train Loss : [0.03197] Val Loss : [0.02058] Val F1 Score : [0.47500]
Epoch 00029: reducing learning rate of group 0 to 3.6621e-06.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[[152   6]
 [ 10   0]]
              precision    recall  f1-score   support

       0: 정상       0.94      0.96      0.95       158
       1: 이상       0.00      0.00      0.00        10

    accuracy                           0.90       168
   macro avg       0.47      0.48      0.47       168
weighted avg       0.88      0.90      0.89       168

Epoch [29], Train Loss : [0.03148] Val Loss : [0.02055] Val F1 Score : [0.47500]
Student Train Best Score는 0.5174603174603175 입니다.
부분학습 종료
----------------------------------------------------------------------
전체 학습 종료
best scores : [0.5825781462453788, 0.5918442370055274, 0.5967116249197174, 0.5174603174603175]


- best scores : [0.5973296328245133, 0.5762557077625571, 0.587967433732445, 0.6592292089249492]


In [694]:
def choose_threshold(model, val_loader, device):
    model.to(device)
    model.eval()
    
#     thresholds = [0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
    thresholds = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

    pred_labels = []
    true_labels = []
    
    best_score = 0
    best_thr = None
    with torch.no_grad():
        for _, x_s, y in tqdm(iter(val_loader)):
            x_s = x_s.float().to(device)
            y = y.float().to(device)
            
            model_pred = model(x_s)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        for threshold in thresholds:
            pred_labels_thr = np.where(np.array(pred_labels) > threshold, 1, 0)
            score_thr = competition_metric(true_labels, pred_labels_thr)
            cf1 = confusion_matrix(true_labels, pred_labels_thr)  

#             print(cf1)
#             print(classification_report(true_labels, pred_labels_thr, target_names=["0: 정상", "1: 이상"]))

            if best_score < score_thr:  
                best_score = score_thr
                best_thr = threshold
                
    return best_thr, best_score

In [695]:
# for a, b in zip(students, s_val_loaders):   
#     print(list(b))
thresholds = []
best_scores = []

for a, b, c, d, e, f in zip(s_train_loaders, s_val_loaders, train_X_list, test_list, teachers, students):
    t_start_col_num = len(c.columns)
    test_stage_features = d.iloc[:,:-1].columns
    s_start_col_num = len(test_stage_features)


    choose_threshold(f, b, device)
    best_threshold, best_score = choose_threshold(f, b, device)
    choose_threshold_result = f'Best Threshold : [{best_threshold}], Score : [{best_score:.5f}]'
    thresholds.append(best_threshold)
    best_scores.append(best_score)
    print(choose_threshold_result) 
    print("부분 작업 종료")
    
print(thresholds)
print(best_scores)
print("전체 작업 종료")

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Best Threshold : [0.25], Score : [0.56759]
부분 작업 종료


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

Best Threshold : [0.3], Score : [0.59231]
부분 작업 종료


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Best Threshold : [0.25], Score : [0.57713]
부분 작업 종료
[0.25, 0.3, 0.3, 0.25]
[0.5675936046834763, 0.5447813152731186, 0.5923145082311244, 0.5771305285868393]
전체 작업 종료


- best scores : [0.5973296328245133, 0.5762557077625571, 0.587967433732445, 0.6592292089249492]


In [696]:
test_2.head(2)

,encoder__x1_2008,encoder__x1_2009,encoder__x1_2010,encoder__x1_2011,encoder__x1_2012,encoder__x1_2013,encoder__x1_2014,encoder__x1_2015,encoder__x1_2016,encoder__x1_2017,encoder__x1_2018,encoder__x1_2019,encoder__x1_2020,encoder__x1_2021,encoder__x1_2022,ANONYMOUS_1,ANONYMOUS_2,MN,order
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.392465,-0.62177,-0.352471,2.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.112354,-0.62177,-0.352471,4.0


In [697]:
test_loaders = []

for i in test_list:

    test_dataset = CustomDataset(i.iloc[:,:-1], None, False)
    test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)
    test_loaders.append(test_loader)

# print(list(test_loaders[0]))
    

In [698]:
def inference(model, test_loader, threshold, device):
    model.to(device)
    model.eval()
    # .eval함수는 evaluation 과정에서 사용하지 말아야 하는 layer들을 알아서 off 시키는 함수
    
    test_predict = []
    with torch.no_grad():
        for x in tqdm(test_loader):
            x = x.float().to(device)
            model_pred = model(x)

            model_pred = model_pred.squeeze(1).to('cpu')
            test_predict += model_pred
        
    test_predict = np.where(np.array(test_predict) > threshold, 1, 0)  
    # threshhold 보다 큰 값을 찾아서 1로 바꾸고, 아닌 것은 0으로 변경해라
    print('Done.')
    return test_predict

In [699]:
pred_list = []
order_list = []
for a, b, c, d in zip(students, test_loaders, test_list, thresholds):
    preds = inference(a, b, d, device)
    pred_list.extend(preds)
    order_list.extend(c.order.tolist())
    

  0%|          | 0/7 [00:00<?, ?it/s]

Done.


  0%|          | 0/4 [00:00<?, ?it/s]

Done.


  0%|          | 0/12 [00:00<?, ?it/s]

Done.


  0%|          | 0/2 [00:00<?, ?it/s]

Done.


In [700]:
{"예측값": pred_list, "순서": order_list}

{'예측값': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [701]:
result = pd.DataFrame({"예측값": pred_list, "순서": order_list})
result.sort_values(by=["순서"], inplace=True)
result

,예측값,순서
0,0,0.0
2660,0,1.0
1667,0,2.0
2661,0,3.0
1668,1,4.0
...,...,...
5678,0,6036.0
5679,0,6037.0
5680,0,6038.0
2659,0,6039.0


In [702]:
final_result = result["예측값"].tolist()
final_result

[0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [703]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['Y_LABEL'] = final_result
submit.head(10)

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,1
5,TEST_0005,0
6,TEST_0006,0
7,TEST_0007,0
8,TEST_0008,0
9,TEST_0009,0


In [704]:
pred_cnt = tuple(submit["Y_LABEL"].value_counts())
a, b = pred_cnt
r = f"정상값 개수 : {a}, 이상값 개수 : {b}"
print(r)

정상값 개수 : 5628, 이상값 개수 : 413


In [705]:
submit.to_csv(f'./submits/submit{file_version}.csv', index=False)

In [706]:
record = {"ver": file_version, "scaler": scaler, "CFG": CFG, "threshold": thresholds, "best_scores": best_scores, "pred_cnt": r}
record

{'ver': 122,
 'scaler': StandardScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.045, 0.05],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.25, 0.3, 0.3, 0.25],
 'best_scores': [0.5675936046834763,
  0.5447813152731186,
  0.5923145082311244,
  0.5771305285868393],
 'pred_cnt': '정상값 개수 : 5628, 이상값 개수 : 413'}

In [707]:
{'ver': 108,
 'scaler': RobustScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.05, 0.05],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.25, 0.3, 0.3, 0.35],
 'best_scores': [0.5680344529579042,
  0.5352564102564102,
  0.593927331993648,
  0.6161993769470405],
 'pred_cnt': '정상값 개수 : 5498, 이상값 개수 : 543'}

{'ver': 108,
 'scaler': RobustScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.05, 0.05],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.25, 0.3, 0.3, 0.35],
 'best_scores': [0.5680344529579042,
  0.5352564102564102,
  0.593927331993648,
  0.6161993769470405],
 'pred_cnt': '정상값 개수 : 5498, 이상값 개수 : 543'}

In [708]:
{'ver': 110,
 'scaler': RobustScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.05, 0.05, 0.05, 0.05],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.25, 0.3, 0.25, 0.2],
 'best_scores': [0.5562327862974854,
  0.5501811316903923,
  0.5894364110003587,
  0.6332225913621263],
 'pred_cnt': '정상값 개수 : 5397, 이상값 개수 : 644'}

{'ver': 110,
 'scaler': RobustScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.05, 0.05, 0.05, 0.05],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.25, 0.3, 0.25, 0.2],
 'best_scores': [0.5562327862974854,
  0.5501811316903923,
  0.5894364110003587,
  0.6332225913621263],
 'pred_cnt': '정상값 개수 : 5397, 이상값 개수 : 644'}

In [709]:
{'ver': 111,
 'scaler': RobustScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.045, 0.045],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.3, 0.3, 0.25, 0.2],
 'best_scores': [0.5923514000884624,
  0.5531923801549089,
  0.5930189629132405,
  0.5899846334628943],
 'pred_cnt': '정상값 개수 : 5546, 이상값 개수 : 495'}

{'ver': 111,
 'scaler': RobustScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.045, 0.045],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.3, 0.3, 0.25, 0.2],
 'best_scores': [0.5923514000884624,
  0.5531923801549089,
  0.5930189629132405,
  0.5899846334628943],
 'pred_cnt': '정상값 개수 : 5546, 이상값 개수 : 495'}

In [710]:
{'ver': 112,
 'scaler': RobustScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.045, 0.05],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.3, 0.35, 0.25, 0.25],
 'best_scores': [0.5631122480813626,
  0.5430715123094959,
  0.6025049465614023,
  0.6493506493506493],
 'pred_cnt': '정상값 개수 : 5564, 이상값 개수 : 477'}

{'ver': 112,
 'scaler': RobustScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.045, 0.05],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.3, 0.35, 0.25, 0.25],
 'best_scores': [0.5631122480813626,
  0.5430715123094959,
  0.6025049465614023,
  0.6493506493506493],
 'pred_cnt': '정상값 개수 : 5564, 이상값 개수 : 477'}

In [711]:
{'ver': 120,
 'scaler': StandardScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.05, 0.05],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.3, 0.25, 0.35, 0.25],
 'best_scores': [0.5767048465620429,
  0.545202724010671,
  0.5767304441249427,
  0.6493506493506493],
 'pred_cnt': '정상값 개수 : 5326, 이상값 개수 : 715'}

{'ver': 120,
 'scaler': StandardScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.05, 0.05],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.3, 0.25, 0.35, 0.25],
 'best_scores': [0.5767048465620429,
  0.545202724010671,
  0.5767304441249427,
  0.6493506493506493],
 'pred_cnt': '정상값 개수 : 5326, 이상값 개수 : 715'}

In [712]:
{'ver': 121,
 'scaler': StandardScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.045, 0.045],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.25, 0.3, 0.25, 0.2],
 'best_scores': [0.5649130074565037,
  0.5285141278188786,
  0.5914518531321008,
  0.5644444444444445],
 'pred_cnt': '정상값 개수 : 5461, 이상값 개수 : 580'}

{'ver': 121,
 'scaler': StandardScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.045, 0.045],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.25, 0.3, 0.25, 0.2],
 'best_scores': [0.5649130074565037,
  0.5285141278188786,
  0.5914518531321008,
  0.5644444444444445],
 'pred_cnt': '정상값 개수 : 5461, 이상값 개수 : 580'}

In [713]:
{'ver': 122,
 'scaler': StandardScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.045, 0.05],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.25, 0.3, 0.3, 0.25],
 'best_scores': [0.5675936046834763,
  0.5447813152731186,
  0.5923145082311244,
  0.5771305285868393],
 'pred_cnt': '정상값 개수 : 5628, 이상값 개수 : 413'}

{'ver': 122,
 'scaler': StandardScaler(),
 'CFG': {'EPOCHS': 30,
  'BATCH_SIZE': 256,
  'SEED': 1,
  'log_transform_turn': 1,
  'skew_cut': 0,
  'vif_cut': 10,
  'corr_cut': [0.045, 0.045, 0.045, 0.05],
  'drop_rate': 0.3,
  'reduct': 'mean',
  'T_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'T_Thresh': [0.3, 0.3, 0.3, 0.3],
  'S_LEARNING_RATE': [0.03, 0.001, 0.01, 0.03],
  'S_Thresh': [0.3, 0.3, 0.3, 0.3],
  'alpha': 0.1},
 'threshold': [0.25, 0.3, 0.3, 0.25],
 'best_scores': [0.5675936046834763,
  0.5447813152731186,
  0.5923145082311244,
  0.5771305285868393],
 'pred_cnt': '정상값 개수 : 5628, 이상값 개수 : 413'}